<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

   ---------------------------------------- 0.0/113.4 kB ? eta -:--:--
   -------------- ------------------------ 41.0/113.4 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 113.4/113.4 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/90.4 kB ? eta -:--:--
   ---------------------------------------- 90.4/90.4 kB 2.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9712 sha256=a95d97e0ae1661158c1d06bd05aa3952c2236a2a06e7d7a0432219d66185837c
  Stored in directory: c:\users\piers\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local\pip\cache\wheels\40\b3\0f\a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [6]:
## Task 1: Mark all launch sites on a map
import requests
import pandas as pd

url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_csv_file = 'spacex_launch_geo.csv'

# Download the file using requests
response = requests.get(url)
if response.status_code == 200:
    with open(spacex_csv_file, 'wb') as f:
        f.write(response.content)
    print(f"File '{spacex_csv_file}' downloaded successfully.")
else:
    print(f"Failed to download file. Status code: {response.status_code}")

# Read the downloaded CSV into a pandas DataFrame
spacex_df = pd.read_csv(spacex_csv_file)

File 'spacex_launch_geo.csv' downloaded successfully.


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [7]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

ModuleNotFoundError: No module named 'js'

Now, you can take a look at what are the coordinates for each site.


In [8]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [9]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [10]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [11]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Start location is NASA Johnson Space Center
import folium
import wget
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

# Descargar y leer el archivo spacex_launch_geo.csv
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

# Seleccionar subcolumnas relevantes: 'Launch Site', 'Lat(Latitude)', 'Long(Longitude)', 'class'
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Coordenadas iniciales para el mapa (NASA Johnson Space Center)
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Crear un círculo azul y un marcador en NASA Johnson Space Center
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)
site_map.add_child(circle)
site_map.add_child(marker)

# Iterar sobre las filas de launch_sites_df y agregar marcadores para cada sitio de lanzamiento
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']

    # Crear un círculo para el sitio de lanzamiento
    circle = folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(site_name))
    site_map.add_child(circle)

    # Crear un marcador para el sitio de lanzamiento
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site_name,
        )
    )
    site_map.add_child(marker)

# Mostrar el mapa (esta línea debe ser la última en la celda para que se renderice en Jupyter)
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [12]:
# Task 2: Mark the success/failed launches for each site on the map
marker_cluster = MarkerCluster()


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [13]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [14]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [16]:

# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10) # Display the last 10 rows to verify the new column

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [17]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed,
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # Create a Marker object
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'],
                         # Optional: add a popup with more info
                         popup=f"Launch Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failure'}"
                        )
    )
    # Add the marker to the marker_cluster
    marker_cluster.add_child(marker)

# Display the map
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [24]:
# TASK 3: Calculate the distances between a launch site to its proximities
# For example, let's use CCAFS SLC 40
# You'll need to retrieve the coordinates from your launch_sites_df
# For demonstration, let's assume these are the coordinates for CCAFS SLC 40
# You should get these from your actual data
launch_site_lat = 28.563197
launch_site_lon = -80.576820
launch_site_coordinates = [launch_site_lat, launch_site_lon]



Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [25]:
# Example for Coastline
coastline_lat = 28.56367
coastline_lon = -80.56799
coastline_coordinates = [coastline_lat, coastline_lon]

# Assume calculate_distance function is defined elsewhere (e.g., from a previous lab part)
# def calculate_distance(lat1, lon1, lat2, lon2):
#     R = 6371 #radius of Earth in KM
#     dLat = math.radians(lat2 - lat1)
#     dLon = math.radians(lon2 - lon1)
#     a = math.sin(dLat / 2) * math.sin(dLat / 2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon / 2) * math.sin(dLon / 2)
#     c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
#     distance = R * c
#     return distance

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a PolyLine object for coastline
lines_coastline = folium.PolyLine(
    locations=[launch_site_coordinates, coastline_coordinates],
    weight=1
)
site_map.add_child(lines_coastline)

# Add a marker at the coastline point
distance_marker_coastline = folium.Marker(
    location=coastline_coordinates,
    icon=DivIcon(
        icon_size=(200,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#d35400;"><b>{round(distance_coastline, 2)} KM</b></div>'
    )
)
site_map.add_child(distance_marker_coastline)

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [26]:

# Example for City (e.g., Melbourne)
city_lat = 28.083
city_lon = -80.608
city_coordinates = [city_lat, city_lon]

distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Create a PolyLine object for city
lines_city = folium.PolyLine(
    locations=[launch_site_coordinates, city_coordinates],
    weight=1,
    color='blue' # Different color for clarity
)
site_map.add_child(lines_city)

# Add a marker at the city point
distance_marker_city = folium.Marker(
    location=city_coordinates,
    icon=DivIcon(
        icon_size=(200,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#228B22;"><b>{round(distance_city, 2)} KM</b></div>'
    )
)
site_map.add_child(distance_marker_city)   

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [27]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# Example for City (e.g., Melbourne)
# Example for Highway (e.g., closest point on SR 3)
# Example for Highway (e.g., closest point on SR 3)
highway_lat = 28.56398
highway_lon = -80.57071
highway_coordinates = [highway_lat, highway_lon]

distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

# Create a PolyLine object for highway
lines_highway = folium.PolyLine(
    locations=[launch_site_coordinates, highway_coordinates],
    weight=1,
    color='purple' # Different color for clarity
)
site_map.add_child(lines_highway)

# Add a marker at the highway point
distance_marker_highway = folium.Marker(
    location=highway_coordinates,
    icon=DivIcon(
        icon_size=(200,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#800080;"><b>{round(distance_highway, 2)} KM</b></div>'
    )
)
site_map.add_child(distance_marker_highway)


In [28]:
# Example for Railway
railway_lat = 28.57207
railway_lon = -80.58529
railway_coordinates = [railway_lat, railway_lon]

distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

# Create a PolyLine object for railway
lines_railway = folium.PolyLine(
    locations=[launch_site_coordinates, railway_coordinates],
    weight=1,
    color='black' # Different color for clarity
)
site_map.add_child(lines_railway)

# Add a marker at the railway point
distance_marker_railway = folium.Marker(
    location=railway_coordinates,
    icon=DivIcon(
        icon_size=(200,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#000000;"><b>{round(distance_railway, 2)} KM</b></div>'
    )
)
site_map.add_child(distance_marker_railway)

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [29]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate# Create a PolyLine object representing a line showcasing the distance
points = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
folium.PolyLine(points, color="blue", weight=2.5, opacity=1).add_to(site_map)
site_map
# lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

NameError: name 'lines' is not defined

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [30]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# Finding the nearest city, highway, and railway to CCAFS SLC-40
# Puedes encontrar estas coordenadas usando Google Maps o datos geoespaciales.

# Coordenadas de la ciudad más cercana (e.g., Cocoa Beach)
city_lat = 28.32283
city_lon = -80.60771
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)
print(f"Distancia a la ciudad más cercana (Cocoa Beach): {distance_city:.2f} km")

# Coordenadas de la autopista más cercana (e.g., A1A)
highway_lat = 28.56342
highway_lon = -80.57071
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)
print(f"Distancia a la autopista más cercana: {distance_highway:.2f} km")

# Coordenadas de la vía férrea más cercana (aproximado)
railway_lat = 28.5720
railway_lon = -80.58529
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)
print(f"Distancia a la vía férrea más cercana: {distance_railway:.2f} km")


Distancia a la ciudad más cercana (Cocoa Beach): 26.91 km
Distancia a la autopista más cercana: 0.60 km
Distancia a la vía férrea más cercana: 1.28 km


In [31]:
# Agregar marcadores y líneas para la ciudad, autopista y vía férrea
# Ciudad
folium.Marker(
    [city_lat, city_lon],
    icon=DivIcon(
        icon_size=(200,200),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
    )
).add_to(site_map)
folium.PolyLine([[launch_site_lat, launch_site_lon], [city_lat, city_lon]], color="purple", weight=2.5, opacity=1).add_to(site_map)

# Autopista
folium.Marker(
    [highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(200,200),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
    )
).add_to(site_map)
folium.PolyLine([[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]], color="orange", weight=2.5, opacity=1).add_to(site_map)

# Vía Férrea
folium.Marker(
    [railway_lat, railway_lon],
    icon=DivIcon(
        icon_size=(200,200),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
    )
).add_to(site_map)
folium.PolyLine([[launch_site_lat, launch_site_lon], [railway_lat, railway_lon]], color="darkgreen", weight=2.5, opacity=1).add_to(site_map)

site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
